In [8]:
import numpy as np
import networkx as nx
# import nxmetis as nxm
# from matplotlib import pyplot as plt
import scipy
# import graphviz as gv
import igraph as ig
# import gurobipy as gp
# from gurobipy import GRB
import pickle
from collections import Counter 


np.random.seed(5)

pngdir = 'pngs/'
txtdir = 'txts/'
pickledir = "middleResults/"
logdir = "gurobiLog/"
uncoarsen = "afteruncoarsen/"
outputdir = "finalpartition/"

# largest_cc = pickle.load(open(pickledir+"cc_cnajk","rb"))
experiment_graphs = pickle.load(open(pickledir+"experiment_graphs","rb"))
# node_map_id2name = pickle.load( open(pickledir+"node_map_id2name-2021-06-02","rb") )

# ig_cnakj = experiment_graphs['cnakj']
gnp_5k = experiment_graphs['5k']['gnp']

In [31]:
dstdir = "/home/data/data12/zhy/codes/topos/experiments/"
fns = ['gnp_5k','pl_5k','as_5k','gnp_1w','pl_1w','as_1w','gnp_5w','pl_5w','as_5w']
# dstdir = pickledir
for fname in fns:
    fn = fname + '.txt'
    coars = open(dstdir+fn,"r")
    alllines = [i.rstrip() for i in coars.readlines()]
    numN = int(alllines[0].split()[0])
    numM = int(alllines[0].split()[1])
    cut = int(alllines[0].split()[2])
    print(numN,numM)
    G = nx.Graph()
    for i in range(1,len(alllines)):
        G.add_node(i, NodeWeight=int(alllines[i].split()[0]))

    for i in range(1,len(alllines)):
        thisline = alllines[i].split()
        for j in range(1,len(thisline),2):
    #         print(i,thisline[j],int(thisline[j+1]))
            G.add_edge(i,int(thisline[j]),EdgeWeight=int(thisline[j+1]))

    assert(list(G.nodes) == list(range(1,numN+1)))
    print(len(G.nodes),len(G.edges))
    NWeight = [G.nodes[i]['NodeWeight'] for i in G.nodes]
    numOriginNodes=5000
#     assert(sum(NWeight) == numOriginNodes)
    EWeight = [G.edges[i]['EdgeWeight'] for i in G.edges]
    assert(sum(EWeight) == cut)
    nowg = G
    node_list = list(nowg.nodes)
    N = len(node_list)
    # N = node_list
    # E = list(nowg.edges)
    # L = nx.laplacian_matrix(nowg)
    W = nx.adj_matrix(nowg,weight='EdgeWeight')

    assert(np.sum(W.data)==cut*2)
    Wary = W.toarray() 


100 4602
100 4602
144 4404
144 4404
732 2135
732 2135
201 15093
201 15093
291 14774
291 14774
550 9083
550 9083
1008 129293
1008 129293
1671 128100
1671 128100
12028 97245
12028 97245


# This is for gurobi output

## always change this line when running

In [11]:
this_solution = "MLP/gnp-5k.txt2021-07-17_74.sol"
NUM_MACHINE = 6
NUM_NODES = 5000
experiment_graphs = pickle.load(open(pickledir+"experiment_graphs","rb"))

NameError: name 'pickle' is not defined

In [35]:
f = open(this_solution, "r")

partition_res = {}

lines = f.readlines()[2:-1]
lines_short = [i.split(' ')[1].rstrip() for i in lines]
id_ary = np.array(lines_short).reshape((NUM_MACHINE,NUM_NODES))
for i in range(NUM_MACHINE):
    for j in range(NUM_NODES):
        if float(id_ary[i,j]) > 0.5:
            partition_res[j] = i
# print(partition_res)
# print(ig_cnakj.vs.get_attribute_values('_nx_name'))
# assert(ig_cnakj.vs.get_attribute_values('_nx_name') == node_list)
assert(len(gnp_5k.vs) == len(partition_res))
# labels = [partition_res[i] for i in ig_cnakj.vs.get_attribute_values('_nx_name')]

labels = [partition_res[i] for i in partition_res]
print(Counter(labels))

vc = ig.VertexClustering(gnp_5k,membership=labels)
contracted = vc.cluster_graph(combine_edges=sum)
contracted.get_adjacency(attribute='weight')

Counter({5: 1000, 1: 812, 0: 806, 3: 801, 2: 799, 4: 782})


Matrix([[0, 1107, 1035, 1006, 959, 1289], [1107, 0, 1071, 1044, 1020, 1322], [1035, 1071, 0, 973, 998, 1240], [1006, 1044, 973, 0, 1064, 1272], [959, 1020, 998, 1064, 0, 1266], [1289, 1322, 1240, 1272, 1266, 0]])

In [39]:
np.array(contracted.get_adjacency(attribute='weight').data).sum()

33332

## write results, notice the output filename

In [32]:
from datetime import date
td = str(date.today())

f = open(outputdir+td+"-ip-obj=","w")
f.writelines(['\''  +  i + '\',' + str(partition_res[i]) + '\n' for i in partition_res])

# This is for kaffpa final output

## always change this line when running

In [66]:
import numpy as np
import networkx as nx
# import nxmetis as nxm
# from matplotlib import pyplot as plt
import scipy
# import graphviz as gv
import igraph as ig
# import gurobipy as gp
# from gurobipy import GRB
import pickle
from collections import Counter 
np.random.seed(5)

pngdir = 'pngs/'
txtdir = 'txts/'
pickledir = "middleResults/"
logdir = "gurobiLog/"
uncoarsen = "afteruncoarsen/"
outputdir = "finalpartition/"
experiment_graphs = pickle.load(open(pickledir+"experiment_graphs","rb"))
exp_2500 = pickle.load(open(pickledir + "exp_2500","rb"))

metisdir = "/home/data/data12/zhy/NetworkPartition/middleResults/formetis/"
kahipdir = "/home/data/data12/zhy/NetworkPartition/middleResults/kahip/"
fns = ['gnp_5k','pl_5k','as_5k','gnp_1w','pl_1w','as_1w','gnp_5w','pl_5w','as_5w']

In [97]:
fatt = pickle.load(open(pickledir+"fatt","rb"))
fatt

{24: <igraph.Graph at 0x7feeb1e1fd50>,
 32: <igraph.Graph at 0x7feeaf02e050>,
 56: <igraph.Graph at 0x7feeaf02e150>}

In [87]:
len(experiment_graphs['1w']['as'].vs)

10000

In [51]:
# for fn in fns:
for fn in [24,32,56]:
    tail = 0
#     fn = fn.replace("_","-")
#     if fn.endswith('5k'):
#         tail = 6
#     if fn.endswith('1w'):
#         tail = 12
#     if fn.endswith('5w'):
#         tail = 60
    if fn == 24:
        tail = 6
    if fn == 32:
        tail = 12
    if fn == 56:
        tail = 60
    assert(tail != 0)
#     this_solution = kahipdir + fn + ".txt-part"
    this_solution = kahipdir + "fattree-k=" + str(fn) + ".txt-part"
    print(this_solution)

    
    f = open(this_solution,"r")
    lines = [i.rstrip() for i in f.readlines()]
    
    res = {}
    for i,j in enumerate(lines):
    #     print(i,j)
#         res[node_map_id2name[i]] = int(j)
        res[i] = int(j)
    
#     this_graph = experiment_graphs[fn[-2:]][fn.split('-')[0]]
    this_graph = fatt[fn]

    for i in this_graph.es:
        i['weight'] = 1
    assert(len(this_graph.vs) == len(res))
#     labels = [res[i] for i in this_graph.vs.get_attribute_values('_nx_name')]
    labels = [res[i] for i in res]

#     print(Counter(labels))

    vc = ig.VertexClustering(this_graph,membership=labels)
    contracted = vc.cluster_graph(combine_edges={'weight' :sum})
    W = contracted.get_adjacency(attribute='weight').data
#     print(W)
    print(np.array(W).max())


/home/data/data12/zhy/NetworkPartition/middleResults/kahip/fattree-k=24.txt-part
285
/home/data/data12/zhy/NetworkPartition/middleResults/kahip/fattree-k=32.txt-part
216
/home/data/data12/zhy/NetworkPartition/middleResults/kahip/fattree-k=56.txt-part
512


In [67]:
exp_2500

{'gnp': <igraph.Graph at 0x7feeb4471550>,
 'pl': <igraph.Graph at 0x7feeb4471450>,
 'as': <igraph.Graph at 0x7feeb4471350>,
 'ft': <igraph.Graph at 0x7feeb1e1f650>}

In [108]:
for name in ['pl','as','gnp','ft']:
# for name in ['fattree']:
    this_solution = kahipdir + name + "-2.5k.txt-part"
#     this_solution = metisdir + name+"-k=24.txt.part.12"


    f = open(this_solution,"r")
    lines = [i.rstrip() for i in f.readlines()]

    res = {}
    for i,j in enumerate(lines):
    #     print(i,j)
    #         res[node_map_id2name[i]] = int(j)
        res[i] = int(j)

    #     this_graph = experiment_graphs[fn[-2:]][fn.split('-')[0]]
#     this_graph = experiment_graphs['5k'][name]
#     this_graph = fatt[24]
    this_graph = exp_2500[name]
    for i in this_graph.es:
        i['weight'] = 1
    assert(len(this_graph.vs) == len(res))
    #     labels = [res[i] for i in this_graph.vs.get_attribute_values('_nx_name')]
    labels = [res[i] for i in res]

    #     print(Counter(labels))

    vc = ig.VertexClustering(this_graph,membership=labels)
    contracted = vc.cluster_graph(combine_edges={'weight' :sum})
    W = contracted.get_adjacency(attribute='weight').data
    W = np.array(W)
    # W.sum()
    print(name,W.max(),W)

pl 149 [[  0 124  87 125 122  89 110 109  52  93 118  68]
 [124   0  96 104 100  92  99 105  53  75 101  56]
 [ 87  96   0  69  83  65  87  86  34  73  87  46]
 [125 104  69   0 136 108 112 119  65  93 118  63]
 [122 100  83 136   0  86 131 125  60  90 117  59]
 [ 89  92  65 108  86   0  98  82  26  69  89  43]
 [110  99  87 112 131  98   0 149  60 116 132  67]
 [109 105  86 119 125  82 149   0  83  97 119  70]
 [ 52  53  34  65  60  26  60  83   0  51  54  29]
 [ 93  75  73  93  90  69 116  97  51   0 120  67]
 [118 101  87 118 117  89 132 119  54 120   0  78]
 [ 68  56  46  63  59  43  67  70  29  67  78   0]]
as 59 [[ 0 59 32 52 40 25 33 23 39 21 21 25]
 [59  0 40 53 35 35 30 27 35 20 28 35]
 [32 40  0 31 17 30  9 17  7 14 14 11]
 [52 53 31  0 40 31 26 32 30 23 25 26]
 [40 35 17 40  0 26 29 11 17 20 12  6]
 [25 35 30 31 26  0  8 17 12 16 11 15]
 [33 30  9 26 29  8  0 12 18 10 12 12]
 [23 27 17 32 11 17 12  0  8  4  8  6]
 [39 35  7 30 17 12 18  8  0  8 11  6]
 [21 20 14 23 20 16 10 


## write results, notice the output filename

In [18]:
from datetime import date
td = str(date.today())

f = open(outputdir+td+"-kaffpa-final-obj=","w")
f.writelines(['\''  +  i + '\',' + str(res[i]) + '\n' for i in res])